#### Import Dependencies

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PayloadSchemaType, PointStruct, SparseVectorParams, Document, Prefetch, FusionQuery
from qdrant_client import models

import pandas as pd
import openai
import cohere

#### Retrieval

In [ ]:
qdrant_client = QdrantClient(url="http://localhost:6333")

In [ ]:
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=[text],
        model=model,
    )
    return response.data[0].embedding

In [ ]:
def retrieve_data(query, qdrant_client, k=5):

    query_embedding = get_embedding(query)

    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-01-hybrid-search",
        prefetch=[
            Prefetch(
                query=query_embedding,
                using="text-embedding-3-small",
                limit=20
            ),
            Prefetch(
                query=Document(
                    text=query,
                    model="qdrant/bm25"
                ),
                using="bm25",
                limit=20
            )
        ],
        query=FusionQuery(fusion="rrf"),
        limit=k,
    )

    retrieved_context_ids = []
    retrieved_context = []
    retrieved_context_ratings = []
    similarity_scores = []

    for result in results.points:
        retrieved_context_ids.append(result.payload["parent_asin"])
        retrieved_context.append(result.payload["description"])
        retrieved_context_ratings.append(result.payload["average_rating"])
        similarity_scores.append(result.score)

    return {
        "retrieved_context_ids": retrieved_context_ids,
        "retrieved_context": retrieved_context,
        "retrieved_context_ratings": retrieved_context_ratings,
        "similarity_scores": similarity_scores,
    }

In [ ]:
query = "Can I get some tablet?"

In [ ]:
results = retrieve_data(query, qdrant_client, k=20)

In [ ]:
results

#### Reranking

In [ ]:
cohere_client = cohere.ClientV2()

In [ ]:
to_rerank = results["retrieved_context"]

In [ ]:
to_rerank

In [ ]:
response = cohere_client.rerank(
    model="rerank-v3.5",
    query=query,
    documents=to_rerank,
    top_n=20
)

In [ ]:
response

In [ ]:
reranked_results = [to_rerank[results.index] for results in response.results]

In [ ]:
reranked_results

In [ ]:
results["retrieved_context"]

In [ ]:
reranked_results

In [ ]:
prompt = f"""
given a chunk returned from the DB and a query return a relevance score that could be used to rerank the pieces of context.

<Chunk>
{Chunk}
</Chunk>

<Query>
{Query}
</Query>"""